In [0]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenet-id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://landing-zone-2@ecomadls.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/ecomdata",
extra_configs = configs)

True

In [0]:
%fs ls '/mnt/ecomdata'

path,name,size,modificationTime
dbfs:/mnt/ecomdata/archive/,archive/,0,1724952117000
dbfs:/mnt/ecomdata/buyers-raw/,buyers-raw/,0,1724658536000
dbfs:/mnt/ecomdata/countries-raw/,countries-raw/,0,1724658545000
dbfs:/mnt/ecomdata/sellers-raw/,sellers-raw/,0,1724658527000
dbfs:/mnt/ecomdata/users-raw/,users-raw/,0,1724952987000


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()

In [0]:
spark

In [0]:
userDF = spark.read.format('parquet').load('/mnt/ecomdata/users-raw/')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-422353836847571>, line 1
----> 1 userDF = spark.read.format('parquet').load('/mnt/ecomdata/users-raw/')

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
    313 elif path is not None:
    314     if type(path) != list:

File /databricks/spark/python/lib/py4j-0.10.9

In [0]:
userDF.show()

+--------------------+----+-----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----------------+-----------+
|      identifierHash|type|    country|language|socialNbFollowers|socialNbFollows|socialProductsLiked|productsListed|productsSold|productsPassRate|productsWished|productsBought|gender|civilityGenderId|civilityTitle|hasAnyApp|hasAndroidApp|hasIosApp|hasProfilePicture|daysSinceLastLogin|seniority|seniorityAsMonths|seniorityAsYears|countryCode|
+--------------------+----+-----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+----------------

In [0]:
userDF.write.format('delta')\
    .mode('overwrite')\
    .save('/mnt/delta/tables/bronze/users')

In [0]:
buyersDF = spark.read.format('parquet').option('header', 'true').option('inferSchema', 'true').load('/mnt/ecomdata/buyers-raw/')

In [0]:
buyersDF.show()

+-----------+------+---------+-------------+------------+----------+---------------+-------------+-----------------+--------------------+----------------------+------------------+-------------------------+---------------------+-------------------+-------------------+------------------+----------------------+----------------------+---------------------+------------------+------------------+-----------------+---------------------+---------------------+--------------------+---------------+------------------+-------------+-------------+----------------+----------------+
|    country|buyers|topbuyers|topbuyerratio|femalebuyers|malebuyers|topfemalebuyers|topmalebuyers|femalebuyersratio|topfemalebuyersratio|boughtperwishlistratio|boughtperlikeratio|topboughtperwishlistratio|topboughtperlikeratio|totalproductsbought|totalproductswished|totalproductsliked|toptotalproductsbought|toptotalproductswished|toptotalproductsliked|meanproductsbought|meanproductswished|meanproductsliked|topmeanproductsbo

In [0]:
buyersDF.write.format('delta')\
    .mode('overwrite')\
    .save('/mnt/delta/tables/bronze/buyers')

In [0]:
sellersDF = spark.read.format('parquet').option('header', 'true').option('inferSchema', 'true').load('/mnt/ecomdata/sellers-raw/')

In [0]:
sellersDF.show()

+---------+------+---------+----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------+-----------+------------------+-------------+-----------+-----------------+-----------------+------------------+
|  country|   sex|nbsellers|meanproductssold|meanproductslisted|meansellerpassrate|totalproductssold|totalproductslisted|meanproductsbought|meanproductswished|meanproductsliked|totalbought|totalwished|totalproductsliked|meanfollowers|meanfollows|percentofappusers|percentofiosusers|     meanseniority|
+---------+------+---------+----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------+-----------+------------------+-------------+-----------+-----------------+-----------------+------------------+
|Allemagne|Female|      116|            4.03|              2.72|             27.33|           

In [0]:
sellersDF.write.format('delta')\
    .mode('overwrite')\
    .save('/mnt/delta/tables/bronze/sellers')

In [0]:
countriesDF = spark.read.format('parquet').option('header', 'true').option('inferSchema', 'true').load('/mnt/ecomdata/countries-raw/')

In [0]:
countriesDF.show()

+-----------+-------+----------+--------------+------------------+---------------------+-------------+-----------+----------------+--------------+----------------+-------------+--------------------+-----------------+----------------------+-------------------+--------------------+---------------------+--------------------+--------------------+---------------+------------------+-------------+-------------+----------------+----------------+
|    country|sellers|topsellers|topsellerratio|femalesellersratio|topfemalesellersratio|femalesellers|malesellers|topfemalesellers|topmalesellers|countrysoldratio|bestsoldratio|toptotalproductssold|totalproductssold|toptotalproductslisted|totalproductslisted| topmeanproductssold|topmeanproductslisted|    meanproductssold|  meanproductslisted|meanofflinedays|topmeanofflinedays|meanfollowers|meanfollowing|topmeanfollowers|topmeanfollowing|
+-----------+-------+----------+--------------+------------------+---------------------+-------------+-----------+--

In [0]:
countriesDF.write.format('delta')\
    .mode('overwrite')\
    .save('/mnt/delta/tables/bronze/countries')